In [ ]:
import requests
from dotenv import load_dotenv
import os
import psycopg

load_dotenv()

In [41]:
dbconn = os.getenv("DBCONN")
conn = psycopg.connect(dbconn)
cur = conn.cursor()

In [ ]:
cur.execute('''
ALTER TABLE financial_times_scaped
ADD sentiment VARCHAR(255);
''')

In [65]:
conn.commit()

In [ ]:
headings = cur.execute('''
SELECT heading FROM financial_times_scaped ORDER BY date DESC;
''').fetchall()

headings = [heading[0] for heading in headings]

In [ ]:
hf_api_key = os.getenv("HF_API_KEY")
url = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"

payload = { "inputs": headings }
headers = { "Authorization": f"Bearer {hf_api_key}" }
response = requests.post(url, headers=headers, json=payload)
print("first response", response)

if response.status_code == 503:
  headers["x-wait-for-model"] = "true"
  response = requests.post(url, headers=headers, json=payload)
  print("second response", response)

In [ ]:
import json

sentiment_results = response.json()
print(json.dumps(sentiment_results, indent=2))


In [ ]:
# check that headings are actually unique
# cur.execute('''
# SELECT heading, COUNT(heading) FROM financial_times_scaped
# GROUP BY heading
# HAVING COUNT(heading) > 1
# ''')

# print(cur.fetchall())

In [ ]:
if len(sentiment_results) == len(headings):
  for i in range(0, len(headings)):
    label = sentiment_results[i][0]["label"]
    cur.execute('''
    UPDATE financial_times_scaped
    SET sentiment = %s
    WHERE heading = %s;
    ''', (label, headings[i]))

conn.commit()

In [66]:
cur.close()
conn.close()

In [ ]:
# reset cursor after error
# cur.execute("ROLLBACK")